In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
!pip install pymorphy2
!pip install catboost
!pip install transformers
!pip install sentencepiece
!pip install torch

     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 10.2 MB/s 
     |████████████████████████████████| 76.4 MB 38 kB/s 
     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 3.3 MB 40.5 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 33.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [51]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

In [52]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score
import catboost
import seaborn as sns
import warnings
import re
import itertools
#from simpletransformers import model
warnings.filterwarnings("ignore")
import os
import matplotlib.pyplot as plt
from copy import deepcopy
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# from datasets import load_dataset, load_metric
import torch.nn as nn
os.environ["TOKENIZERS_PARALLELISM"] = "false"
set_seed(42)

#Обработка комментариев


In [53]:
#Принимает на вход строку комментария и преобразует её в список причин и комментарий к водителю (если то и то есть вообще)
def parse_comment(comment):
  if comment[-1] == ';':
    comment = comment[:-1]
    
  comment = comment.replace('\n', '')
  arr = comment[9:]
  passanger_comment = ""
  causes = []

  if "Комментарий" in arr and "Причина:" in arr:
    arr = arr.split("Причина:")
    passanger_comment = arr[0][13:]
    causes = arr[1][1:].split('; ')

  elif "Комментарий" in arr:
    passanger_comment = arr[13:]

  if "Причина:" in arr:
    causes = arr[9:].split('; ')  
  
  return passanger_comment, causes

#Получает на вход датасет и выдает все типы причин, которые встречались
def get_all_categories(comments):
  all_categories = set()
  for i in range(comments.shape[0]):
    for cause in parse_comment(comments["comment"][i])[1]:
      prov = True
      for k in all_categories:
        if cause in k:
          prov = False
          break
      if prov:
        all_categories.add(cause)
  return all_categories

#Принимает на вход датафрейм с комментами словарь из категорий,  
#то есть тк данные были записаны в разное время появляются одни и те же категорий
#записанные в разной форме, например, "Водитель был без маски" и "Водитель не надел маску"
#return: датафрейм с новыми колонками - комментарий к заказу и причины, закодированные one-hot encoding вектором
def get_categories_for_data(comments, all_categories_dict):
  passanger_comments = pd.DataFrame()

  for cause in set(all_categories_dict.values()):
    passanger_comments[cause] = [0] * comments.shape[0]
  passanger_comments["passanger_comment"] = [""] * comments.shape[0]
  passanger_comments["id_order"] = comments["id_order"]
  
  for i in tqdm(range(passanger_comments.shape[0])):

    if len(comments["comment"][i]) <= 11:
      continue
    passanger_comment, causes = parse_comment(comments["comment"][i])
    
    for cause in causes:
      prov = True
      for k in all_categories_dict.values():
        if cause in k:
          passanger_comments[all_categories_dict[k]][i] = 1
          break
    passanger_comments["passanger_comment"][i] = passanger_comment

  return passanger_comments    

In [6]:
all_categories = {'В машине плохо пахло': 'В машине плохо пахло',
 'Водитель был без маски': 'Водитель был без маски',
 'Водитель был невежлив': 'Водитель был невежлив',
 'Водитель вежливый': 'Водитель вежливый',
 'Водитель знает город': 'Водитель знает город',
 'Водитель не надел маску': 'Водитель был без маски',
 'Водитель опасно вёл': 'Неаккуратное вождение',
 'Знание города': 'Водитель знает город',
 'Классная музыка': 'Классная музыка',
 'Комфортная поездка': 'Комфортная поездка',
 'Манера вождения': 'Манера вождения',
 'Машина ниже классом, чем ожидалось': 'Машина ниже классом, чем ожидалось',
 'Не тот номер автомобиля': 'Номер машины отличался',
 'Неаккуратное вождение': 'Неаккуратное вождение',
 'Небезопасное вождение': 'Неаккуратное вождение',
 'Номер машины отличался': 'Номер машины отличался',
 'Отлично водит': 'Отлично водит',
 'Поведение водителя': 'Поведение водителя',
 'Подача машины не туда': 'Подача машины не туда',
 'Помощь и забота': 'Помощь и забота',
 'Приятное общение': 'Приятное общение',
 'Проблемы с приложением': 'Проблемы с приложением',
 'Работа приложения': 'Работа приложения',
 'Сложности с приложением': 'Проблемы с приложением',
 'Состояние машины': 'Состояние машины',
 'Стоимость поездки': 'Стоимость поездки',
 'Хорошо поговорили': 'Приятное общение',
 'Чистота машины': 'Чистота машины'}

In [ ]:
comments = pd.read_csv("/content/drive/MyDrive/final/df_cl_rates_on_rides.csv")

In [47]:
prepeared_comments = pd.read_csv("/content/drive/MyDrive/prepeared_comments.csv", index_col=0)
#prepeared_comments = get_categories_for_data(comments, all_categories)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
all_drives = pd.read_csv("/content/drive/MyDrive/final/df_ride_data.csv")

In [48]:
#возвращает комментарии, которые имеют хотя бы одну причину или комментарий пассажира
def get_comments_with_causes(passanger_comments):
  order_id_with_comments = []
  passanger_comments.index = passanger_comments["id_order"]
  for i in tqdm(range(passanger_comments.shape[0])):
    if  passanger_comments["Водитель был невежлив"][i] == 1 or passanger_comments["Поведение водителя"][i] == 1 or \
        passanger_comments["Отлично водит"][i] == 1 or passanger_comments["Водитель вежливый"][i] == 1 or \
        passanger_comments["Проблемы с приложением"][i] == 1 or passanger_comments["Подача машины не туда"][i] == 1 or \
        passanger_comments["Манера вождения"][i] == 1 or passanger_comments["Помощь и забота"][i] == 1 or \
        passanger_comments["Водитель был без маски"][i] == 1 or passanger_comments["Стоимость поездки"][i] == 1 or \
        passanger_comments["Классная музыка"][i] == 1 or passanger_comments["Водитель знает город"][i] == 1 or \
        passanger_comments["Комфортная поездка"][i] == 1 or passanger_comments["Приятное общение"][i] == 1 or \
        passanger_comments["Состояние машины"][i] == 1 or passanger_comments["Машина ниже классом, чем ожидалось"][i] == 1 or \
        passanger_comments["Чистота машины"][i] == 1 or passanger_comments["Работа приложения"][i] == 1 or \
        passanger_comments["В машине плохо пахло"][i] == 1 or passanger_comments["Неаккуратное вождение"][i] == 1 or \
        passanger_comments["Номер машины отличался"][i] == 1 or passanger_comments["passanger_comment"][i] != "":

      order_id_with_comments.append(passanger_comments["id_order"][i])
  usefull_comments = passanger_comments.loc[order_id_with_comments]
  return usefull_comments

#возвращает словарь, в котором ключ это месяц и год, 
#а значение это словарь из дней этого месяца и количеством поездок в этот день
def get_day_with_count_rides(data):
    months = set()
    dic = {}
    for i in tqdm(data["dt_15_min"]):
      if not dic.get(i[:7]):
          dic[i[:7]] = {}
      
      prov = True
      days = dic[i[:7]]
      
      for day in days.keys():
        if i[:10] == day:
          dic[i[:7]][day] += 1
          prov = False
          break
      if prov:
        dic[i[:7]][i[:10]] = 1
    return dic

In [180]:
selection_data = pd.read_csv("/content/drive/MyDrive/V_rate.csv")

In [54]:
normal_comments = get_comments_with_causes(prepeared_comments)

100%|██████████| 3238726/3238726 [06:21<00:00, 8485.77it/s]


In [ ]:
normal_comments.shape

(561572, 25)

In [ ]:
len(set(normal_comments["id_order"]) & set(all_drives["id_order"]))

561572

In [ ]:
print("Весомых комментариев (%):", normal_comments.shape[0] / prepeared_comments.shape[0] * 100)

Весомых комментариев (%): 17.339287114748206


In [ ]:
count = 0
for i in tqdm(range(normal_comments.shape[0])):
  if normal_comments["passanger_comment"][i] != "":
    count += 1
print()
print("Личных комментариев пасссажира (%):", count / prepeared_comments.shape[0] * 100)

100%|██████████| 561572/561572 [00:03<00:00, 168686.75it/s]


Личных комментариев пасссажира: 1.9536076840090826


In [178]:
#добавляет в data преобразованные комментарии с помощью функции parse_comment
#если закинуть сначала комментарии в get_comments_with_causes, будет работать гораздо быстрее
def add_comments_inf(data, passanger_comments, causes):
  causes = set(causes.values())
  for cause in causes - set("passanger_comment"):
    data[cause] = [0] * data.shape[0]
  data["passanger_comment"] = [""] * data.shape[0]
  
  passanger_comments.index = passanger_comments["id_order"]
  data.index = data["id_order"]
  id_orders = set(passanger_comments.index) & set(data["id_order"])
  passanger_comments = passanger_comments.loc[id_orders]

  for order_id in tqdm(id_orders):
    for cause in causes:
      data[cause][order_id] = passanger_comments.loc[order_id][cause]
    data["passanger_comment"][order_id] = passanger_comments.loc[order_id]["passanger_comment"]
  data.index = range(data.shape[0]) 
  return data

In [ ]:
selection_data = add_comments_inf(selection_data, prepeared_comments, all_categories)

# XLM

In [ ]:
comment = pd.read_csv("drive/MyDrive/Data_rus/final_all_comments.csv")
X_driver_comment = comment[["comment", "Резкие маневры"]]
X_driver_comment.columns=["comment", "label"]
X_car_comment = comment[["comment","Проблемы с транспортом"]]
X_car_comment.columns=["comment", "label"]
X_behavior_comment = comment[["comment", "Агрессивное поведение"]]
X_behavior_comment.columns=["comment", "label"]

In [79]:
class SentenceClassifier(nn.Module):

    def __init__(self, bert_model="sismetanin/xlm_roberta_large-ru-sentiment-rureviews", freeze_bert=False):
        super(SentenceClassifier, self).__init__()
        #  Instantiating BERT-based model object
        self.bert = AutoModel.from_pretrained(bert_model)
        # Freeze bert layers and only train the classification layer weights
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False
        
        self.bert_model = bert_model
        
        if self.bert_model == "sismetanin/rubert-toxic-pikabu-2ch":
          D_in, H, D_out = 768, 512, 1
        elif self.bert_model == "sismetanin/xlm_roberta_large-ru-sentiment-rureviews":
          D_in, H, D_out = 1024, 512, 1

        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Linear(H, D_out)
        )

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids):

        outputs = self.bert(input_ids, attn_masks, token_type_ids)
        
        last_hidden_state_cls = outputs[0][:, 0, :]
        
        logits = self.classifier(last_hidden_state_cls)

        return logits

In [80]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model="sismetanin/xlm_roberta_large-ru-sentiment-rureviews"):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model)  

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent = str(self.data.loc[index, 'comment'])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.maxlen,  
                                      return_tensors='pt', 
                                      return_token_type_ids=True)  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'label']
            return token_ids, attn_masks, token_type_ids, label
        else:
            return token_ids, attn_masks, token_type_ids

In [81]:
def evaluate_loss(net, device, criterion, dataloader):
    net.eval()

    mean_loss = 0
    count = 0
    mean_acc = 0

    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)

            fin_targets.extend(labels.float().cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(logits.float()).cpu().detach().numpy().tolist())
            count += 1
    
    lst_out = []
    for i in range(len(fin_targets)):
      ans = 0
      ans = fin_outputs[i][0]
      lst_out.append(ans)
      fin_targets[i] = int(fin_targets[i])
    return roc_auc_score(fin_targets, lst_out)

In [82]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = -1
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []

    scaler = GradScaler()

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:

                scaler.step(opti)

                scaler.update()

                lr_scheduler.step()

                opti.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


    del loss
    torch.cuda.empty_cache() 

In [ ]:
def get_loader(data, bert_model, test_size, maxlen, batch_size):
    if batch_size == 0:
      train_comment_loader = DataLoader(CustomDataset(data, maxlen, with_labels=True, bert_model=bert_model), batch_size = batch_size, num_workers = 5)
      return train_comment_loader, _
      
    else:
      train_comment, val_comment, _, _ = train_test_split(data, data, test_size = test_size, stratify = data["label"])

      train_comment.index = range(train_comment.shape[0])
      val_comment.index = range(val_comment.shape[0])

      train_comment_loader = DataLoader(CustomDataset(train_comment, maxlen, with_labels=True, bert_model=bert_model), batch_size = batch_size, num_workers = 5)
      val_comment_loader = DataLoader(CustomDataset(val_comment, maxlen, with_labels=True, bert_model=bert_model), batch_size = batch_size, num_workers = 5)

    return train_comment_loader, val_comment_loader


test_size = 0.3
train_driver_comment_loader, val_driver_comment_loader = get_loader(X_driver_comment, bert_model, test_size, maxlen, bs)

train_car_comment_loader, val_car_comment_loader = get_loader(X_car_comment, bert_model, test_size, maxlen, bs)

train_behavior_comment_loader, val_behavior_comment_loader = get_loader(X_behavior_comment, bert_model, test_size, maxlen, bs)

Downloading:   0%|          | 0.00/540 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/920 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [85]:
bert_model = "sismetanin/rubert-toxic-pikabu-2ch"  # 'albert-base-v2', 'albert-large-v2', 'albert-xlarge-v2', 'albert-xxlarge-v2', 'bert-base-uncased', ...
freeze_bert = False  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = 150  # maximum length of the tokenized input sentence pair : if greater than "maxlen", the input is truncated and else if smaller, the input is padded
bs = 4  # batch size
iters_to_accumulate = 1  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = 0.000005  # learning rate
epochs = 3  # number of training epochs
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
model_behavior = SentenceClassifier(bert_model).to(device)
criterion = nn.BCEWithLogitsLoss()
opti = AdamW(model_behavior.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_car_comment_loader)  # The total number of training steps
t_total = (len(train_car_comment_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_bert(model_behavior, criterion, opti, lr, lr_scheduler, train_behavior_comment_loader, val_behavior_comment_loader, epochs, iters_to_accumulate)

Some weights of the model checkpoint at sismetanin/rubert-toxic-pikabu-2ch were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 20%|█▉        | 50/253 [00:30<02:00,  1.69it/s]


Iteration 50/253 of epoch 1 complete. Loss : 0.6242398172616959 


 40%|███▉      | 100/253 [00:59<01:30,  1.69it/s]


Iteration 100/253 of epoch 1 complete. Loss : 0.474338920712471 


 59%|█████▉    | 150/253 [01:29<01:00,  1.69it/s]

...

 79%|███████▉  | 200/253 [01:59<00:31,  1.69it/s]


Iteration 200/253 of epoch 3 complete. Loss : 0.19598325166851283 


 99%|█████████▉| 250/253 [02:28<00:01,  1.69it/s]


Iteration 250/253 of epoch 3 complete. Loss : 0.21168798282742501 


100%|██████████| 253/253 [02:30<00:00,  1.68it/s]


In [86]:
#bert_model = "sismetanin/xlm_roberta_large-ru-sentiment-rureviews"
#model_driver = SentenceClassifier(bert_model).to(device)
#model_driver.load_state_dict(torch.load('/content/drive/MyDrive/cause_nets/net_driver.pt'))# if you want load pretrained model
#model_car = SentenceClassifier(bert_model).to(device)
#model_car.load_state_dict(torch.load('/content/drive/MyDrive/cause_nets/net_car.pt'))# if you want load pretrained model
#model_behavior = SentenceClassifier(bert_model).to(device)
#model_behavior.load_state_dict(torch.load('/content/drive/MyDrive/cause_nets/net_behavior.pt'))# if you want load pretrained model

bert_model = "sismetanin/rubert-toxic-pikabu-2ch"
model_driver = SentenceClassifier(bert_model).to(device)
model_driver.load_state_dict(torch.load('drive/MyDrive/sismetanin\rubert-toxic-pikabu-2ch_net_driver.pt'))# if you want load pretrained model
model_car = SentenceClassifier(bert_model).to(device)
model_car.load_state_dict(torch.load('drive/MyDrive/sismetanin\rubert-toxic-pikabu-2ch_net_car.pt'))# if you want load pretrained model
model_behavior = SentenceClassifier(bert_model).to(device)
model_behavior.load_state_dict(torch.load('drive/MyDrive/sismetanin\rubert-toxic-pikabu-2ch_net_behavior.pt'))# if you want load pretrained model

Some weights of the model checkpoint at sismetanin/rubert-toxic-pikabu-2ch were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at sismetanin/rubert-toxic-pikabu-2ch were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
-

<All keys matched successfully>

In [ ]:
evaluate_loss(model_driver, device, criterion, val_driver_comment_loader)

100%|██████████| 109/109 [00:20<00:00,  5.40it/s]


0.9581314354936402

In [ ]:
evaluate_loss(model_behavior, device, criterion, val_behavior_comment_loader)

100%|██████████| 109/109 [00:19<00:00,  5.62it/s]


0.9495696611081227

In [ ]:
evaluate_loss(model_car, device, criterion, val_car_comment_loader)

100%|██████████| 109/109 [00:19<00:00,  5.65it/s]


0.9462962962962963

In [ ]:
net_copy = deepcopy(model_behavior) 
path_to_model = 'drive/MyDrive/sismetanin\rubert-toxic-pikabu-2ch_net_behavior.pt'
torch.save(net_copy.state_dict(), path_to_model)

# Дополнительная обработка

In [134]:
#поступают на вход данные с поездками, 3 модели определения причины, и максимальная длина
#возвращает данные с новыми столбцами вероятностей причин плохой поездки по комментарию, если он есть
def get_probability_comment(data, model_driver, model_behavior, model_car, max_len):

  data["probability_driver"] = [0.0] * data.shape[0]
  data["probability_behavior"] = [0.0] * data.shape[0]
  data["probability_car"] = [0.0] * data.shape[0]
  data["comment"] = data["passanger_comment"]
  comments = CustomDataset(data, maxlen = maxlen, with_labels=False, bert_model=bert_model)
  for it in tqdm(range(data.shape[0])):
    if data["passanger_comment"][it] != "":
      seq, attn_masks, token_type_ids = comments[it]
      
      seq, attn_masks, token_type_ids  = seq.to(device).reshape(1, -1), attn_masks.to(device).reshape(1, -1), token_type_ids.to(device).reshape(1, -1)

      probability_driver = torch.sigmoid(model_driver(seq, attn_masks, token_type_ids)).cpu().detach().numpy()
      probability_behavior = torch.sigmoid(model_behavior(seq, attn_masks, token_type_ids)).cpu().detach().numpy()
      probability_car = torch.sigmoid(model_car(seq, attn_masks, token_type_ids)).cpu().detach().numpy()

      if probability_driver > 0.5:
        data["probability_driver"][it]   = probability_driver
      if probability_behavior > 0.5:
        data["probability_behavior"][it] = probability_behavior
      if probability_car > 0.5:
        data["probability_car"][it]      = probability_car
  data = data.drop(["comment"], 1)
  return data 

In [150]:
selection_data = get_probability_comment(selection_data, model_driver, model_behavior, model_car, maxlen)

100%|██████████| 554936/554936 [03:09<00:00, 2922.32it/s]


In [131]:
def get_day_week(arr):
    lst = []
    for i in arr:
        lst.append(int(i.weekday()))
    return lst

def get_day(arr):
    lst = []
    for i in arr:
        if int(i.month) == 1:
            lst.append(int(i.day))
        elif int(i.month) == 2:
            lst.append(int(i.day) + 31)
        elif int(i.month) == 3:
            lst.append(int(i.day) + 59)
        elif int(i.month) == 4:
            lst.append(int(i.day) + 90)
        elif int(i.month) == 5:
            lst.append(int(i.day) + 120)
        elif int(i.month) == 6:
            lst.append(int(i.day) + 151)
        elif int(i.month) == 7:
            lst.append(int(i.day) + 181)
        elif int(i.month) == 8:
            lst.append(int(i.day) + 212)
        elif int(i.month) == 9:
            lst.append(int(i.day) + 242)
        elif int(i.month) == 10:
            lst.append(int(i.day) + 273)
        elif int(i.month) == 11:
            lst.append(int(i.day) + 303)
    return lst

def get_small_day(arr):
    lst = []
    for i in arr:
        lst.append(int(i.day))
    return lst
def get_year(arr):
    lst = []
    for i in arr:
        lst.append(int(i.year))
    return lst

def get_month(arr):
    lst = []
    for i in arr:
        lst.append(int(i.month))
    return lst

def get_minute(arr):
    lst = []
    for i in arr:
        lst.append(i.hour * 60 + i.minute)
    return lst

def get_hour(arr):
    lst = []
    for i in arr:
        lst.append(i.hour)
    return lst


In [164]:
#считает отношение агрессивных поездок ко всем поездкам по дням недели и по часам за последнее количество дней (count_day_before), показывая сложность дня
#примечание: можно сделать пересечение дней с часами, то есть давать не за весь день или не за час любого дня, а за определенный час в дне 
def get_time_situation(data, year, month, big_day, day, week_day, hour, count_day_before):
  
  today_day = big_day
  today_week_day = week_day
  data = data[(data.day >= today_day - count_day_before) & (data.day <= today_day)]
  mean_mark_hours = []
  mean_mark_days = []

  for i in range(24):
    a = np.mean(data[data.hour == i]["is_agressive_proba"]) * 100
    mean_mark_hours.append(a)
  
  for i in range(7):
    b = np.mean(data[data.day_week == i]["is_agressive_proba"]) * 100
    mean_mark_days.append(b)
  return np.mean(mean_mark_days), mean_mark_days[today_week_day], np.mean(mean_mark_days), mean_mark_hours[hour]

#получает на вход data с поездками, id водителя, и количество поездок 
#возвращает среднее по последним n поездкам водителя его:
#усредненные вероятности по комментариям к предыдущим комментам за n дней
def get_driver_comment_situation(data, id_driver, today, day_count_before):

    data = data[(data["id_driver"] == id_driver) & (data["day"] > today - count_day_before) & (data["day"] < today)]
    probability_drivers = data[data["probability_driver"] != 0]
    probability_behaviors = data[data["probability_behavior"] != 0]
    probability_cars = data[data["probability_car"] != 0]
    
    mean_driver = np.mean(probability_drivers["probability_driver"])
    mean_behavior = np.mean(probability_behaviors["probability_behavior"])
    mean_car = np.mean(probability_cars["probability_car"])
        
    return mean_driver, mean_behavior, mean_car

# Get_K_rate

In [ ]:
selection_data["day"] = get_day(selection_data["dtm_15_min"])

In [ ]:
selection_data = selection_data.merge(all_rides["dtm_15_min"], left_on='id_driver', right_on='id_driver')

In [2]:
import pandas as pd

In [166]:
selection_data["aggressive_proba"] = data_with_aggressive["aggressive_proba"]

In [168]:
data_with_aggressive["mean_behavior"] = [0.0] * data_with_aggressive.shape[0]
data_with_aggressive["mean_driver"] = [0.0] * data_with_aggressive.shape[0]
data_with_aggressive["mean_car"] = [0.0] * data_with_aggressive.shape[0]

In [ ]:
for i in tqdm(range(selection_data.shape[0])):
  selection_data["mean_driver"][i], selection_data["mean_car"][i], selection_data["mean_behavior"][i] = get_driver_comment_situation(selection_data, selection_data["id_client"][i], selection_data["day"][i], 10)

In [174]:
selection_data["K_rate"] = [0.0] * selection_data.shape[0]
for i in tqdm(range(selection_data.shape[0])):
  selection_data["K_rate"][i] = (1 - selection_data["aggressive_proba"][i]) * selection_data["driver_rate_ride"][i] * (1 - (selection_data["mean_behavior"][i] + selection_data["mean_driver"][i] + selection_data["mean_car"][i]))

In [ ]:
selection_data.to_csv("K_rate.csv")